# TPS-Feb-2022

In [1]:
NB = '007'

## Import libralies

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style='white', context='notebook', palette='deep')

In [7]:
from math import factorial

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans

## Load and check data

### Load data

In [3]:
# Load data
##### Load train and Test set
train_df = pd.read_csv("../data/raw/train.csv")
test_df = pd.read_csv("../data/raw/test.csv")

In [4]:
FEATURES = [col for col in train_df.columns if col not in ['row_id', 'target']]

## Feature Engineering
- 平均、分散、最大値、最小値、中央値、第一四分位、第三四分位、尖度、歪度を追加

In [5]:
train_df["mean"] = train_df[FEATURES].mean(axis=1)
train_df["std"] = train_df[FEATURES].std(axis=1)
train_df["min"] = train_df[FEATURES].min(axis=1)
train_df["max"] = train_df[FEATURES].max(axis=1)
train_df["median"] = train_df[FEATURES].median(axis=1)
train_df["25%"] = train_df[FEATURES].quantile(q=0.25, axis=1)
train_df["75%"] = train_df[FEATURES].quantile(q=0.75, axis=1)
train_df["skew"] = train_df[FEATURES].skew(axis=1)
train_df["kurt"] = train_df[FEATURES].kurt(axis=1)

test_df["mean"] = test_df[FEATURES].mean(axis=1)
test_df["std"] = test_df[FEATURES].std(axis=1)
test_df["min"] = test_df[FEATURES].min(axis=1)
test_df["max"] = test_df[FEATURES].max(axis=1)
test_df["median"] = test_df[FEATURES].median(axis=1)
test_df["25%"] = test_df[FEATURES].quantile(q=0.25, axis=1)
test_df["75%"] = test_df[FEATURES].quantile(q=0.75, axis=1)
test_df["skew"] = test_df[FEATURES].skew(axis=1)
test_df["kurt"] = test_df[FEATURES].kurt(axis=1)

# FEATURES.extend(['mean', 'std', 'min', 'max', 'median', '25%', '75%', 'skew', 'kurt', 'gcd'])

## 　主成分分析

In [16]:
pca = PCA()
pca.fit(train_df[FEATURES])
train_pca = pca.transform(train_df[FEATURES])

train_pca_cols = ["PCA{}".format(x + 1) for x in range(len(train_df[FEATURES].columns))]
train_pca_df = pd.DataFrame(train_pca, columns=train_pca_cols)

train_pca_df.head()

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA277,PCA278,PCA279,PCA280,PCA281,PCA282,PCA283,PCA284,PCA285,PCA286
0,0.006646,0.069854,-0.023743,0.018470,-0.004203,0.009313,-0.011676,0.001996,0.015399,-0.004787,...,1.850972e-05,-0.000008,-5.585292e-06,1.196883e-05,0.000006,-1.944287e-06,0.000002,5.485936e-07,1.329793e-07,9.540979e-18
1,-0.025797,0.006773,-0.008334,-0.003302,-0.008506,-0.011347,0.004151,0.001914,0.000588,0.005052,...,-1.870797e-05,-0.000017,6.318468e-06,-4.761966e-06,0.000001,-6.246728e-07,-0.000003,3.177374e-07,-1.351602e-07,2.428613e-17
2,-0.029070,-0.005515,0.000657,-0.001592,0.002292,0.001942,-0.002271,-0.000323,-0.001389,0.000822,...,9.390363e-06,0.000007,-1.643514e-07,-6.017212e-07,0.000005,1.573900e-06,-0.000004,1.036072e-06,9.360844e-07,-8.153200e-17
3,-0.032686,-0.006226,0.005941,0.003985,0.002150,0.000738,-0.001066,-0.001300,-0.000586,0.000742,...,-1.077665e-06,-0.000005,4.160533e-07,-3.928258e-06,-0.000002,-5.688576e-07,-0.000005,-8.481520e-08,-1.877031e-07,-1.006140e-16
4,0.042883,0.053586,0.009960,-0.011809,0.040478,-0.015330,0.005708,0.011352,-0.000899,-0.015670,...,-5.375752e-07,0.000005,6.200262e-05,1.986051e-06,0.000008,4.731725e-07,0.000001,-2.807576e-06,4.226935e-07,3.295975e-17


In [18]:
train_exp_df = pd.DataFrame(pca.explained_variance_ratio_, index=train_pca_cols)

In [19]:
train_exp_df.cumsum()

,0
PCA1,0.241967
PCA2,0.390881
PCA3,0.436054
PCA4,0.466916
PCA5,0.486691
PCA6,0.500958
PCA7,0.512850
PCA8,0.524118
PCA9,0.534561
PCA10,0.544481


In [15]:
pca = PCA()
pca.fit(test_df[FEATURES])
test_pca = pca.transform(test_df[FEATURES])

test_pca_cols = ["PCA{}".format(x + 1) for x in range(len(test_df[FEATURES].columns))]
test_pca_df = pd.DataFrame(test_pca, columns=test_pca_cols)

test_pca_df.head()

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA277,PCA278,PCA279,PCA280,PCA281,PCA282,PCA283,PCA284,PCA285,PCA286
0,-0.026061,-0.005676,-0.002087,-0.004364,0.001970,-0.003346,0.001415,-0.000043,0.000773,0.000364,...,0.000003,-3.079238e-07,6.099493e-07,0.000011,6.532714e-06,8.414694e-08,-5.418227e-07,4.001424e-08,-4.094528e-07,-1.101549e-16
1,-0.029970,-0.000829,0.003157,-0.005445,-0.007700,0.006919,-0.004859,-0.001958,0.000154,0.001523,...,-0.000089,-5.040642e-05,5.575077e-06,-0.000016,-1.167959e-06,-1.464624e-06,1.073612e-07,4.598498e-09,-4.829779e-09,-3.295975e-17
2,-0.020501,-0.005960,-0.004886,-0.007022,0.002234,-0.003713,0.002252,0.000387,0.001689,0.000570,...,0.000011,2.294285e-06,7.029154e-07,0.000017,8.773039e-06,-2.242007e-07,-4.700031e-07,1.585401e-07,4.145179e-07,-8.239937e-18
3,-0.032448,-0.006448,0.005718,0.003875,0.002457,-0.000752,0.001256,-0.001193,0.000631,0.000712,...,-0.000012,-9.835746e-07,-2.447849e-07,-0.000009,-3.755113e-07,-2.461793e-07,-1.023788e-06,7.705193e-07,-1.224953e-07,-1.110223e-16
4,0.030404,-0.004855,0.002239,0.004519,-0.004469,0.009123,-0.000260,-0.000754,0.003835,-0.002049,...,-0.000019,5.582952e-06,-3.521718e-06,-0.000014,-4.237364e-07,-1.709658e-06,6.245489e-07,-2.008912e-07,-8.452278e-08,-2.775558e-17


In [ ]:
pd.DataFrame(pca.explained_variance_ratio_, index=test_pca_cols)

### マージ

In [24]:
train_df = pd.concat(objs=[train_df, train_pca_df], axis=1)
test_df = pd.concat(objs=[test_df, test_pca_df], axis=1)

## Save dataset

In [25]:
pd.to_pickle(train_df, f"../data/processed/nb{NB}_train.pkl", compression='zip')
pd.to_pickle(test_df, f"../data/processed/nb{NB}_test.pkl", compression='zip')

# 検証メモ

In [ ]:
dataset[dataset['target'] == 'Streptococcus_pyogenes'].describe()

In [ ]:
train_df.columns.get_loc('A4T4G1C1')

In [ ]:
train_df.columns.get_loc(train_df[FEATURES].idxmax(axis=1))

In [13]:
train_pca_df

array([[ 6.64632144e-03,  6.98539913e-02, -2.37427587e-02, ...,
         5.48593563e-07,  1.32979319e-07,  9.54097912e-18],
       [-2.57972780e-02,  6.77285417e-03, -8.33373794e-03, ...,
         3.17737442e-07, -1.35160166e-07,  2.42861287e-17],
       [-2.90703790e-02, -5.51456053e-03,  6.56628899e-04, ...,
         1.03607184e-06,  9.36084444e-07, -8.15320034e-17],
       ...,
       [-1.48741811e-02, -6.85031862e-03, -5.92601717e-03, ...,
         7.10393828e-07,  4.33723747e-08, -5.81132364e-17],
       [-1.77393351e-02,  4.05500766e-03, -1.26390295e-02, ...,
         1.33825779e-07, -1.31453992e-07,  1.08420217e-17],
       [-5.03014223e-02, -6.21804184e-03,  1.96070266e-02, ...,
        -6.37139310e-07, -4.23215429e-07, -4.33680869e-17]])

In [ ]:
train.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train['target'].unique()

In [ ]:
train.columns

### Targetのバランス
- 結論：ほぼ均等

In [ ]:
dataset['target'].value_counts()

In [ ]:
target_df = pd.DataFrame(train['target'].value_counts()).reset_index()
target_df.columns = ['target', 'count']
target_df['percentage'] = target_df['count'] / len(train) * 100

target_df

In [ ]:
g = sns.catplot(x="count", y="target", data=target_df, kind="bar")
g = g.set_ylabels("Num of target")